# Project : CabEdge

## This file lists all the functions that was used for webscrapping and APIs for the Cabedge project

### Creating a function which effectively gets the Lat and Long from nominatim

In [6]:
import pandas as pd
import requests
import json
import googlemaps

In [16]:
import import_ipynb

In [17]:
import cabedge_utils

importing Jupyter notebook from cabedge_utils.ipynb


In [566]:
#     osm_query = 'q=Queens+Jamaica+Bay&format=json'

def get_lat_lon(df,name):
    osm_url = ' https://nominatim.openstreetmap.org/search?'
    name_list=[]

    if('/' not in name):
        new_name = name.replace('_','+')
    else:
        new_name = name.replace('_','+')
        name_list = name.split('/')
        new_name = name_list[0]
        print(new_name)
        
    osm_query = 'q='+new_name + '&format=json'
    final_query = osm_url+osm_query
    print(final_query)
    response = requests.get(final_query)
    print(response.status_code)
    data = response.json()

    return data

In [ ]:
latitude=[]
longitude =[]
for row in zone_df.index:
    data = get_lat_lon(zone_df,zone_df.loc[row,'New_Name'])
    try:
        lat = data[0]['lat']
        lon = data[0]['lon']
    except:
        lat=0
        lon=0
    latitude.append(lat)
    longitude.append(lon)
    print(lat,lon)
        
print('Latitude :',latitude)
print('Longitude: ',longitude)

### Using Google Maps API to get Lat and Lon for places which were missed by Nominatim

In [572]:
gmaps = googlemaps.Client(key = '')

In [573]:
zone_df[(zone_df['Latitude'] == 0) & (zone_df['Longitude'] == 0)].index

Int64Index([  3,   5,   6,  15,  23,  27,  28,  44,  47,  65,  66,  69,  71,
             72,  77,  83,  84,  89,  96, 103, 104, 105, 115, 117, 118, 121,
            135, 144, 145, 146, 154, 155, 158, 167, 168, 172, 186, 193, 200,
            204, 206, 207, 208, 212, 213, 214, 220, 224, 229, 230, 231, 232,
            233, 235, 241, 242, 246, 248, 250, 254, 256, 259],
           dtype='int64', name='LocationID')

In [574]:
for row in zone_df[(zone_df['Latitude'] == 0) & (zone_df['Longitude'] == 0)].index:
    search_string = 'NewYork ' + zone_df.iloc[row-1,0] + ' ' + zone_df.iloc[row-1,1]
    ### Do not run this code
#     geocode_result = gmaps.geocode(search_string)
    zone_df.iloc[row-1,4]=geocode_result[0]['geometry']['location']['lat']
    zone_df.iloc[row-1,5]=geocode_result[0]['geometry']['location']['lng']
    print(search_string)
    print(zone_df.iloc[row-1,4], zone_df.iloc[row-1,5])

NewYork Bronx Allerton/Pelham Gardens
40.8627726 -73.84343919999999
NewYork Staten Island Arden Heights
40.556413 -74.1735044
NewYork Staten Island Arrochar/Fort Wadsworth
40.6012117 -74.0579185
NewYork Queens Bay Terrace/Fort Totten
40.7922167 -73.7760641
NewYork Staten Island Bloomfield/Emerson Hill
40.6074525 -74.0963115
NewYork Queens Breezy Point/Fort Tilden/Riis Beach
40.5616379 -73.8953425
NewYork Queens Briarwood/Jamaica Hills
40.7109315 -73.81356099999999
NewYork Staten Island Charleston/Tottenville
40.5055264 -74.254178
NewYork Bronx Claremont/Bathgate
40.84128339999999 -73.9001573
NewYork Brooklyn Downtown Brooklyn/MetroTech
40.6930987 -73.98566339999999
NewYork Brooklyn DUMBO/Vinegar Hill
40.7030556 -73.9847221
NewYork Bronx East Concourse/Concourse Village
40.8228141 -73.91998800000002
NewYork Brooklyn East Flatbush/Farragut
40.6374022 -73.9298718
NewYork Brooklyn East Flatbush/Remsen Village
40.6466536 -73.9131066
NewYork Brooklyn East New York/Pennsylvania Avenue
40.6566

### Building a function to get weather details per hour everyday

In [12]:
api_key = 'API_KEY'
month_day = {'jan' : 31,'Feb': 28, 'Mar':31,'Apr':30,'May':31,'Jun':30,'Jul':31,'Aug':31,'Sep':30,'Oct':31,'Nov':30,'Dec':31}
month_list = list(month_day.keys())

def get_weather_data():
    weather_data_1 = pd.DataFrame()
    month_index=1
    for month in month_list:
        for day in range(1,month_day[month]+1):
            if(month_index <= 9):
                month_string = '0'+ str(month_index)
            else:
                month_string = str(month_index)
                
            """Total URL"""
            if(day <= 9):
                """Add a string 0"""
                dark_sky_url = 'https://api.darksky.net/forecast/'+api_key+'/40.6971494,-74.2598617,2018-'+month_string+'-0'+str(day)+'T00:00:00+00:00?exclude=currently,flags'
            else:
                dark_sky_url = 'https://api.darksky.net/forecast/'+api_key+'/40.6971494,-74.2598617,2018-'+month_string+'-'+str(day)+'T00:00:00+00:00?exclude=currently,flags'
            
            weather_response = requests.get(dark_sky_url)
            
            print(dark_sky_url)
            print(weather_response.status_code)
            
            data = weather_response.json()
            
            """Retrieving the horuly data"""
            weather_data_2 = pd.DataFrame(data['hourly']['data'])
            
            """Concatanating the DataFrame"""
            weather_data_1 = pd.concat([weather_data_1,weather_data_2],sort=False)
                
        month_index += 1
    
    try:
        weather_data_1.rename(columns={'time':'datetime'},inplace=True)
    except:
        pass
        
    return weather_data_1

In [13]:
# weather_data_2018 = get_weather_data()

https://api.darksky.net/forecast/f4cd8a991243b80b4ff359dcfc372bf2/40.6971494,-74.2598617,2018-01-01T00:00:00+00:00?exclude=currently,flags
200
https://api.darksky.net/forecast/f4cd8a991243b80b4ff359dcfc372bf2/40.6971494,-74.2598617,2018-01-02T00:00:00+00:00?exclude=currently,flags
200
https://api.darksky.net/forecast/f4cd8a991243b80b4ff359dcfc372bf2/40.6971494,-74.2598617,2018-01-03T00:00:00+00:00?exclude=currently,flags
200
https://api.darksky.net/forecast/f4cd8a991243b80b4ff359dcfc372bf2/40.6971494,-74.2598617,2018-01-04T00:00:00+00:00?exclude=currently,flags
200
https://api.darksky.net/forecast/f4cd8a991243b80b4ff359dcfc372bf2/40.6971494,-74.2598617,2018-01-05T00:00:00+00:00?exclude=currently,flags
200
https://api.darksky.net/forecast/f4cd8a991243b80b4ff359dcfc372bf2/40.6971494,-74.2598617,2018-01-06T00:00:00+00:00?exclude=currently,flags
200
https://api.darksky.net/forecast/f4cd8a991243b80b4ff359dcfc372bf2/40.6971494,-74.2598617,2018-01-07T00:00:00+00:00?exclude=currently,flags
200

In [14]:
weather_data_2018.shape

(8760, 19)

In [23]:
weather_data_2018.head()

,apparentTemperature,cloudCover,dewPoint,humidity,icon,precipIntensity,precipProbability,pressure,summary,temperature,datetime,uvIndex,visibility,windBearing,windGust,windSpeed,precipAccumulation,precipType,ozone
0,18.61,0.21,11.57,0.74,clear-night,0.0,0.0,1018.0,Clear,18.61,2017-12-31 16:00:00,0.0,9.937,280.0,4.12,1.52,NaN,NaN,NaN
1,11.74,0.14,11.32,0.74,clear-night,0.0,0.0,1018.5,Clear,18.12,2017-12-31 17:00:00,0.0,9.997,279.0,6.41,4.22,NaN,NaN,NaN
2,11.26,0.09,9.38,0.70,clear-night,0.0,0.0,1018.8,Clear,17.53,2017-12-31 18:00:00,0.0,9.997,276.0,6.59,4.07,NaN,NaN,NaN
3,10.41,0.08,8.33,0.68,clear-night,0.0,0.0,1019.9,Clear,17.01,2017-12-31 19:00:00,0.0,9.997,276.0,6.50,4.25,NaN,NaN,NaN
4,10.99,0.00,7.35,0.67,clear-night,0.0,0.0,1020.7,Clear,16.38,2017-12-31 20:00:00,0.0,9.997,293.0,6.93,3.38,NaN,NaN,NaN


In [18]:
weather_data_2018['datetime']= weather_data_2018['datetime'].apply(cabedge_utils.unix_to_utc)

In [20]:
weather_data_2018['datetime'] = pd.to_datetime(weather_data_2018['datetime'])

In [21]:
weather_data_2019 = pd.read_csv('C:/Users/Arinab/Desktop/DataScience/capstone/Processed-dataframe/weather_data_utc_time.csv')

In [22]:
weather_data_2019.rename(columns={'time':'datetime'},inplace=True)

In [24]:
weather_data_2019.head()

,Unnamed: 0,apparentTemperature,cloudCover,dewPoint,humidity,icon,ozone,precipAccumulation,precipIntensity,precipProbability,precipType,pressure,summary,temperature,datetime,uvIndex,visibility,windBearing,windGust,windSpeed
0,0,32.53,0.61,28.49,0.85,partly-cloudy-night,297.0,0.0012,0.0002,0.01,snow,1025.5,Mostly Cloudy,32.53,2018-12-31 16:00:00,0,10.0,249,3.11,2.40
1,1,32.60,0.73,28.15,0.83,partly-cloudy-night,298.4,NaN,0.0000,0.00,NaN,1025.8,Mostly Cloudy,32.60,2018-12-31 17:00:00,0,10.0,228,3.18,2.54
2,2,31.40,0.74,28.18,0.88,partly-cloudy-night,296.4,0.0116,0.0016,0.02,snow,1026.6,Mostly Cloudy,31.40,2018-12-31 18:00:00,0,10.0,192,1.98,1.67
3,3,31.17,0.84,28.08,0.88,partly-cloudy-night,293.8,NaN,0.0000,0.00,NaN,1027.2,Mostly Cloudy,31.17,2018-12-31 19:00:00,0,10.0,318,2.22,1.53
4,4,31.69,0.72,28.40,0.87,partly-cloudy-night,290.4,0.0037,0.0006,0.01,snow,1027.2,Mostly Cloudy,31.69,2018-12-31 20:00:00,0,10.0,254,2.53,1.54


In [25]:
weather_data = pd.concat([weather_data_2018,weather_data_2019],ignore_index=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [26]:
weather_data.shape

(17520, 20)

In [28]:
weather_data

,Unnamed: 0,apparentTemperature,cloudCover,datetime,dewPoint,humidity,icon,ozone,precipAccumulation,precipIntensity,precipProbability,precipType,pressure,summary,temperature,uvIndex,visibility,windBearing,windGust,windSpeed
0,NaN,18.61,0.21,2017-12-31 16:00:00,11.57,0.74,clear-night,NaN,NaN,0.0000,0.00,NaN,1018.0,Clear,18.61,0.0,9.937,280.0,4.12,1.52
1,NaN,11.74,0.14,2017-12-31 17:00:00,11.32,0.74,clear-night,NaN,NaN,0.0000,0.00,NaN,1018.5,Clear,18.12,0.0,9.997,279.0,6.41,4.22
2,NaN,11.26,0.09,2017-12-31 18:00:00,9.38,0.70,clear-night,NaN,NaN,0.0000,0.00,NaN,1018.8,Clear,17.53,0.0,9.997,276.0,6.59,4.07
3,NaN,10.41,0.08,2017-12-31 19:00:00,8.33,0.68,clear-night,NaN,NaN,0.0000,0.00,NaN,1019.9,Clear,17.01,0.0,9.997,276.0,6.50,4.25
4,NaN,10.99,0.00,2017-12-31 20:00:00,7.35,0.67,clear-night,NaN,NaN,0.0000,0.00,NaN,1020.7,Clear,16.38,0.0,9.997,293.0,6.93,3.38
5,NaN,8.26,0.08,2017-12-31 21:00:00,6.77,0.67,clear-night,NaN,NaN,0.0000,0.00,NaN,1021.7,Clear,15.90,0.0,9.997,310.0,6.92,4.92
6,NaN,5.77,0.16,2017-12-31 22:00:00,4.63,0.63,clear-night,NaN,NaN,0.0000,0.00,NaN,1022.9,Clear,15.10,0.0,9.997,313.0,9.49,6.33
7,NaN,3.08,0.21,2017-12-31 23:00:00,0.97,0.56,clear-night,NaN,NaN,0.0000,0.00,NaN,1024.0,Clear,13.83,0.0,9.997,317.0,12.70,7.58
8,NaN,1.54,0.33,2018-01-01 00:00:00,-0.67,0.54,partly-cloudy-day,NaN,NaN,0.0000,0.00,NaN,1024.1,Partly Cloudy,12.75,0.0,9.997,325.0,10.37,7.83
9,NaN,3.08,0.34,2018-01-01 01:00:00,-0.79,0.53,partly-cloudy-day,NaN,NaN,0.0000,0.00,NaN,1025.4,Partly Cloudy,13.15,0.0,9.997,313.0,14.17,6.69


In [29]:
weather_data.drop(columns=['Unnamed: 0'],inplace=True)

In [30]:
weather_data.to_csv('C:/Users/Arinab/Desktop/DataScience/capstone/Processed-dataframe/weather_data.csv')